In [103]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [111]:
import pandas as pd
import numpy as np

df_pesoEaltura = pd.read_csv("/content/drive/MyDrive/Data/BDIPMamaV11-INTELIPesoEAltura_DATA_LABELS_2023-01-24_1926.csv")

In [112]:
df_pesoEaltura = df_pesoEaltura.rename(columns={"Record ID": "record_id",
                                                "Data: ": "data",
                                                "Peso": "peso",
                                                "Altura (em centimetros)": "altura"})

lista_RecorId = df_pesoEaltura['record_id'].unique()
df_novo = pd.DataFrame()

for i in lista_RecorId: 
    df_query = df_pesoEaltura.query(f'record_id == {i}')
    df_query = df_query.sort_values(by='data')

    df_row_first = df_query.iloc[0]

    if len(df_query) > 1: 
      df_row_second = df_query.iloc[1]

      numero = len(df_query) - 1

      for col in reversed(['altura', 'peso', 'altura']):
        while not(pd.isna(df_row_first[col].all())) and not(pd.isna(df_row_second[col].all())) and numero > 1: 
          df_row_second = df_query.iloc[len(df_query) - numero]
          numero -= 1

      df_row_first['ultimo_peso'] = df_row_second['peso']
      df_row_first['ultima_altura'] = df_row_second['altura']
      df_row_first['ultimo_IMC'] = df_row_second['IMC']
      
      df_novo = df_novo.append(df_row_first, ignore_index=True)
      
    else: 
      df_novo = df_novo.append(df_row_first, ignore_index=True)

df_novo = df_novo.rename(columns={
                        "peso": "primeiro_peso", 
                        "altura": "primeira_altura",
                        "IMC": "primeiro_IMC"
                        })

df_pesoEaltura = df_novo

/usr/local/lib/python3.8/dist-packages/pandas/core/series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()
/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1724: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer_missing(indexer, value)


In [113]:
np.random.seed(73)

df_pesoEaltura['primeiro_IMC'] = (df_pesoEaltura['primeiro_peso'] / pow(df_pesoEaltura['primeira_altura']/100, 2))
df_pesoEaltura['ultimo_IMC'] = (df_pesoEaltura['ultimo_peso'] / pow(df_pesoEaltura['ultima_altura']/100, 2))

primeiro_peso_mean = df_pesoEaltura["primeiro_peso"].mean(skipna=True)
primeiro_peso_std = df_pesoEaltura["primeiro_peso"].std(skipna=True)
null_primeiro_peso = df_pesoEaltura.primeiro_peso.isnull().sum()
lista_normal_primeiro_peso = np.random.normal(primeiro_peso_mean, primeiro_peso_std, null_primeiro_peso)

primeira_altura_mean = df_pesoEaltura["primeira_altura"].mean(skipna=True)
primeira_altura_std = df_pesoEaltura["primeira_altura"].std(skipna=True)
null_primeira_altura = df_pesoEaltura.primeira_altura.isnull().sum()
lista_normal_primeira_altura = np.random.normal(primeira_altura_mean, primeira_altura_std, null_primeira_altura)

ultimo_peso_mean = df_pesoEaltura["ultimo_peso"].mean(skipna=True)
ultimo_peso_std = df_pesoEaltura["ultimo_peso"].std(skipna=True)
null_ultimo_peso = df_pesoEaltura.ultimo_peso.isnull().sum()
lista_normal_ultimo_peso = np.random.normal(ultimo_peso_mean, ultimo_peso_std, null_ultimo_peso)

ultima_altura_mean = df_pesoEaltura["ultima_altura"].mean(skipna=True)
ultima_altura_std = df_pesoEaltura["ultima_altura"].std(skipna=True)
null_ultima_altura = df_pesoEaltura.ultima_altura.isnull().sum()
lista_normal_ultima_altura = np.random.normal(ultima_altura_mean, ultima_altura_std, null_ultima_altura)

for i in range(0, null_primeiro_peso):
  lista_normal_primeiro_peso[i] = round(lista_normal_primeiro_peso[i])

for i in range(0, null_primeira_altura):
  lista_normal_primeira_altura[i] = round(lista_normal_primeira_altura[i])

for i in range(0, null_ultimo_peso):
  lista_normal_ultimo_peso[i] = round(lista_normal_ultimo_peso[i])

for i in range(0, null_ultima_altura):
  lista_normal_ultima_altura[i] = round(lista_normal_ultima_altura[i])

df_pesoEaltura["primeiro_peso"] = df_pesoEaltura["primeiro_peso"].fillna(pd.Series(lista_normal_primeiro_peso, index=df_pesoEaltura["primeiro_peso"].index[df_pesoEaltura["primeiro_peso"].isnull()]))
df_pesoEaltura["primeira_altura"] = df_pesoEaltura["primeira_altura"].fillna(pd.Series(lista_normal_primeira_altura, index=df_pesoEaltura["primeira_altura"].index[df_pesoEaltura["primeira_altura"].isnull()]))
df_pesoEaltura["ultimo_peso"] = df_pesoEaltura["ultimo_peso"].fillna(pd.Series(lista_normal_ultimo_peso, index=df_pesoEaltura["ultimo_peso"].index[df_pesoEaltura["ultimo_peso"].isnull()]))
df_pesoEaltura["ultima_altura"] = df_pesoEaltura["ultima_altura"].fillna(pd.Series(lista_normal_ultima_altura, index=df_pesoEaltura["ultima_altura"].index[df_pesoEaltura["ultima_altura"].isnull()]))

df_pesoEaltura

,record_id,Repeat Instrument,Repeat Instance,data,primeiro_peso,primeira_altura,primeiro_IMC,ultimo_peso,ultima_altura,ultimo_IMC
0,302.0,Dados Antropometricos,20.0,2009-01-02,57.80,158.0,23.153341,64.90,158.0,25.997436
1,710.0,Dados Antropometricos,18.0,2008-07-12,61.00,156.0,25.065746,63.80,155.0,26.555671
2,752.0,Dados Antropometricos,2.0,2009-12-03,84.00,152.0,36.357341,85.90,152.0,37.179709
3,1367.0,Dados Antropometricos,1.0,2010-04-28,57.90,143.0,28.314343,51.00,143.0,24.940095
4,1589.0,Dados Antropometricos,12.0,2009-04-03,63.20,167.0,22.661264,63.65,167.0,22.822618
...,...,...,...,...,...,...,...,...,...,...
4267,82123.0,Dados Antropometricos,5.0,2021-02-07,85.40,153.0,36.481695,78.00,154.0,NaN
4268,82124.0,Dados Antropometricos,1.0,2020-11-10,81.10,151.0,35.568615,78.00,151.0,34.209026
4269,82131.0,Dados Antropometricos,1.0,2020-11-01,71.40,156.5,29.152079,75.65,156.0,31.085634
4270,82205.0,Dados Antropometricos,4.0,2021-02-28,135.85,174.0,44.870525,78.00,174.0,NaN


In [114]:
df_pesoEaltura['diferenca_peso'] = df_pesoEaltura['ultimo_peso'] - df_pesoEaltura['primeiro_peso']
df_pesoEaltura['coef_peso'] = df_pesoEaltura['ultimo_peso'] / df_pesoEaltura['primeiro_peso']
df_pesoEaltura['diferenca_altura'] = df_pesoEaltura['ultima_altura'] - df_pesoEaltura['primeira_altura']

df_pesoEaltura

,record_id,Repeat Instrument,Repeat Instance,data,primeiro_peso,primeira_altura,primeiro_IMC,ultimo_peso,ultima_altura,ultimo_IMC,diferenca_peso,coef_peso,diferenca_altura
0,302.0,Dados Antropometricos,20.0,2009-01-02,57.80,158.0,23.153341,64.90,158.0,25.997436,7.10,1.122837,0.0
1,710.0,Dados Antropometricos,18.0,2008-07-12,61.00,156.0,25.065746,63.80,155.0,26.555671,2.80,1.045902,-1.0
2,752.0,Dados Antropometricos,2.0,2009-12-03,84.00,152.0,36.357341,85.90,152.0,37.179709,1.90,1.022619,0.0
3,1367.0,Dados Antropometricos,1.0,2010-04-28,57.90,143.0,28.314343,51.00,143.0,24.940095,-6.90,0.880829,0.0
4,1589.0,Dados Antropometricos,12.0,2009-04-03,63.20,167.0,22.661264,63.65,167.0,22.822618,0.45,1.007120,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4267,82123.0,Dados Antropometricos,5.0,2021-02-07,85.40,153.0,36.481695,78.00,154.0,NaN,-7.40,0.913349,1.0
4268,82124.0,Dados Antropometricos,1.0,2020-11-10,81.10,151.0,35.568615,78.00,151.0,34.209026,-3.10,0.961776,0.0
4269,82131.0,Dados Antropometricos,1.0,2020-11-01,71.40,156.5,29.152079,75.65,156.0,31.085634,4.25,1.059524,-0.5
4270,82205.0,Dados Antropometricos,4.0,2021-02-28,135.85,174.0,44.870525,78.00,174.0,NaN,-57.85,0.574163,0.0
